In [15]:
import polars as pl
import os
from datetime import datetime, timezone
import re
from helper_functions import clean_parquet_files

def preprocess_review(review):
    # Remove non-European characters
    review = re.sub(r'[^\x00-\x7F]', '', review)
    # Remove newline and carriage return characters
    review = review.replace('\n', '').replace('\r', '')
    return review

def preprocess_gamereviews(gamereviews_dict):
    # Preprocess the review
    gamereviews_dict["review"] = preprocess_review(gamereviews_dict.get("review", ""))
    
    # Skip rows with reviews less than 20 characters
    if len(gamereviews_dict["review"]) < 20:
        return None
    
    # Format timestamps
    gamereviews_dict["timestamp_created"] = datetime.fromtimestamp(gamereviews_dict.get("timestamp_created", 0), tz=timezone.utc)
    gamereviews_dict["timestamp_updated"] = datetime.fromtimestamp(gamereviews_dict.get("timestamp_updated", 0), tz=timezone.utc)
    
    # Ensure author field is not None
    author = gamereviews_dict.get("author", {})
    
    # Unpack nested author fields
    gamereviews_dict["user_steamid"] = author.get("steamid", "")
    gamereviews_dict["user_num_games_owned"] = author.get("num_games_owned", 0)
    gamereviews_dict["user_num_reviews"] = author.get("num_reviews", 0)
    gamereviews_dict["user_playtime_forever"] = author.get("playtime_forever", 0)
    
    # Remove unnecessary fields
    gamereviews_dict.pop("author", None)
    gamereviews_dict.pop("author_last_played", None)
    gamereviews_dict.pop("language", None)
    gamereviews_dict.pop("hidden_in_steam_china", None)
    gamereviews_dict.pop("steam_china_location", None)
    
    # Format boolean and float fields
    gamereviews_dict["voted_up"] = bool(gamereviews_dict.get("voted_up", False))
    gamereviews_dict["steam_purchase"] = bool(gamereviews_dict.get("steam_purchase", False))
    gamereviews_dict["received_for_free"] = bool(gamereviews_dict.get("received_for_free", False))
    gamereviews_dict["written_during_early_access"] = bool(gamereviews_dict.get("written_during_early_access", False))
    gamereviews_dict["weighted_vote_score"] = float(gamereviews_dict.get("weighted_vote_score", 0.0))
    
    # Convert back to a DataFrame row
    return gamereviews_dict

def preprocess_parquet_file(filepath, output_filepath, appid):
    # Read the parquet file
    df = pl.read_parquet(filepath)
    
    # Apply preprocessing to each row
    preprocessed_data = [preprocess_gamereviews(row) for row in df.iter_rows(named=True) if row is not None]
    
    # Filter out None values (reviews less than 20 characters)
    preprocessed_data = [row for row in preprocessed_data if row is not None]
    
    # Create a new DataFrame from preprocessed data
    if preprocessed_data:
        preprocessed_df = pl.DataFrame(preprocessed_data)
        
        # Add the appid column
        preprocessed_df = preprocessed_df.with_columns(pl.lit(appid).alias('appid'))
        
        # Check for duplicates
        preprocessed_df = preprocessed_df.unique()
        
        # Write the processed file
        preprocessed_df.write_parquet(output_filepath)

input_folder = 'data/parquets'
output_folder = 'data/parquets_preprocessed'

clean_parquet_files(input_folder)

for file_name in os.listdir(input_folder):
    match = re.match(r"(\d+)_reviews_\d+\.parquet", file_name)
    if match:
        appid = match.group(1)
        input_filepath = os.path.join(input_folder, file_name)
        output_filepath = os.path.join(output_folder, file_name.replace('.parquet', '_preprocessed.parquet'))
        
        # Check if the processed file already exists
        if not os.path.exists(output_filepath):
            preprocess_parquet_file(input_filepath, output_filepath, appid)


Files kept:
Kept: 1000760_reviews_490.parquet with count: 490
Kept: 1011190_reviews_547.parquet with count: 547
Kept: 1012110_reviews_447.parquet with count: 447
Kept: 1027820_reviews_260.parquet with count: 260
Kept: 104600_reviews_422.parquet with count: 422
Kept: 1061180_reviews_520.parquet with count: 520
Kept: 1062960_reviews_567.parquet with count: 567
Kept: 1064460_reviews_1176.parquet with count: 1176
Kept: 1065310_reviews_3042.parquet with count: 3042
Kept: 1069740_reviews_3270.parquet with count: 3270
Kept: 1070710_reviews_5875.parquet with count: 5875
Kept: 1075740_reviews_1133.parquet with count: 1133
Kept: 1077290_reviews_457.parquet with count: 457
Kept: 1082430_reviews_9192.parquet with count: 9192
Kept: 1085660_reviews_71741.parquet with count: 71741
Kept: 1086640_reviews_616.parquet with count: 616
Kept: 1086940_reviews_335231.parquet with count: 335231
Kept: 1088790_reviews_2014.parquet with count: 2014
Kept: 1091500_reviews_44700.parquet with count: 44700
Kept: 10919